# Emulating the DRP Catalog with a Simple Analytic Model

_Ji Won Park, Phil Marshall_

Created: July 19, 2019 at the LSST DESC hack day
Last run: 2019-07-19

The goals for this demo notebook are to:

* Show what the `Analytic` model class does, and 
* Check that its outputs are sensible.

### Requirements

For this notebook to run to completion, you will need a copy of the test object dataset, and to have installed the dependencies.

In [1]:
# ! pip install -r requirements.txt

In [ ]:
! curl -O 

In [ ]:
import models
import solver

trainval_data = DerpData()
val_data = 

analytic = models.Analytic()
mean, logvar = analytic(X_val)
sample = solver.sample(mean, logvar)
